In [2]:
import numpy as np
import pandas as pd

In [19]:
nvda_sentiment = pd.read_csv('nvda.csv')
nvda_sentiment = nvda_sentiment.drop(['source', 'link', 'Unnamed: 0', 'Unnamed: 0.1'], axis = 1)
nvda_sentiment['Date'] = pd.to_datetime(nvda_sentiment['datetime']).dt.date
nvda_sentiment.drop('datetime', axis = 1)

/var/folders/s2/04l93vjs1sd427p8mgfj27fh0000gn/T/ipykernel_76021/3529677599.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  nvda_sentiment['Date'] = pd.to_datetime(nvda_sentiment['datetime']).dt.date


,title,sentiment_label,sentiment_score,Date
0,"5 Top Stock Trades for Tuesday: NVDA, TSLA, GILD",Neutral,0.999999,2019-11-25
1,Stock Market Today: Merger Monday; Tesla’s Cyb...,Neutral,0.999994,2019-11-25
2,10 Fantastic Tech Gifts to Buy for $500 and Up,Positive,0.985829,2019-11-26
3,"Shares Could Go Higher, but Nvidia Stock Is Ov...",Negative,0.991712,2019-11-26
4,"Tuesday’s Vital Data: Nvidia, Bank of America ...",Neutral,0.999958,2019-11-26
...,...,...,...,...
8944,Quantum announces support for ‘NVIDIA GPUDirec...,Positive,0.998504,2024-12-18
8945,"4 stocks to watch on Wednesday: GIS, MU and more",Neutral,0.999953,2024-12-18
8946,Notable open interest changes for December 18th,Neutral,0.999996,2024-12-18
8947,Early notable gainers among liquid option name...,Positive,0.940363,2024-12-18


In [20]:
daily_sentiment = nvda_sentiment.groupby('Date').agg({
    'sentiment_label' : lambda x : x.value_counts().to_dict(),
    'sentiment_score': 'mean'
}).reset_index()

def net_sentiment_score(row):
    sentiment_counts = row['sentiment_label']
    net_score = (sentiment_counts.get('Positive', 0)) - (sentiment_counts.get('Negative', 0))
    total_counts = sum(sentiment_counts.values())
    if total_counts == 0:
        return 0
    else: 
        return net_score / total_counts

In [21]:
daily_sentiment['Net Sentiment Score'] = daily_sentiment.apply(net_sentiment_score, axis = 1)
daily_sentiment['Volume_Sentiment Score'] = daily_sentiment['Net Sentiment Score'] * daily_sentiment['article_count']

In [22]:
daily_sentiment.to_csv('nvda_sentiment_aggregated_new.csv', index = False)

In [29]:
import pandas as pd
import numpy as np

def aggregate_daily_sentiment(headlines_df, output_path):
    """
    Aggregate headlines to daily sentiment features and export to CSV.
    """
    # Convert datetime to date for daily aggregation
    headlines_df = headlines_df.copy()
    headlines_df['date'] = pd.to_datetime(headlines_df['datetime']).dt.date
    
    # Group by date
    daily_agg = headlines_df.groupby('date').agg({
        'sentiment_label': 'count',
        'sentiment_score': ['mean', 'std']
    }).reset_index()
    
    # Flatten column names
    daily_agg.columns = ['date', 'article_count', 'avg_sentiment', 'sentiment_std']
    
    # Calculate positive/negative ratio per day
    sentiment_counts = headlines_df.groupby(['date', 'sentiment_label']).size().unstack(fill_value=0)
    daily_agg['pos_neg_ratio'] = (sentiment_counts['Positive'] + 1) / (sentiment_counts['Negative'] + 1)
    
    # Handle NAs for rolling calculations
    # Fill NAs in avg_sentiment with forward fill, then backward fill for any remaining
    daily_agg['avg_sentiment'] = daily_agg['avg_sentiment'].fillna(method='ffill').fillna(method='bfill')
    
    # Now calculate moving average and momentum
    daily_agg['sentiment_ma5'] = daily_agg['avg_sentiment'].rolling(window=5, min_periods=1).mean()
    daily_agg['sentiment_momentum'] = daily_agg['avg_sentiment'].diff().fillna(0)  # Fill first day momentum with 0
    
    # Calculate relative volume with min_periods=1
    daily_agg['relative_volume'] = daily_agg['article_count'] / daily_agg['article_count'].rolling(window=20, min_periods=1).mean()
    
    # Fill any remaining NAs in std with 0 (days with single article)
    daily_agg['sentiment_std'] = daily_agg['sentiment_std'].fillna(0)
    
    # Convert date to datetime
    daily_agg['date'] = pd.to_datetime(daily_agg['date'])
    
    # Export to CSV
    daily_agg.to_csv(output_path, index=False)
    
    return daily_agg

# Read and process the data
nvda_sentiment = pd.read_csv('nvda.csv')
aggregated_sentiment = aggregate_daily_sentiment(nvda_sentiment, 'daily_sentiment.csv')

/var/folders/s2/04l93vjs1sd427p8mgfj27fh0000gn/T/ipykernel_41244/2482045671.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  headlines_df['date'] = pd.to_datetime(headlines_df['datetime']).dt.date
/var/folders/s2/04l93vjs1sd427p8mgfj27fh0000gn/T/ipykernel_41244/2482045671.py:27: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily_agg['avg_sentiment'] = daily_agg['avg_sentiment'].fillna(method='ffill').fillna(method='bfill')


In [18]:
nvda_sentiment = nvda_sentiment.dropna()

In [19]:
nvda_sentiment

,date,article_count,avg_sentiment,sentiment_std,pos_neg_ratio,sentiment_ma5,sentiment_momentum,relative_volume
